In [1]:
import sys, numpy as np
from keras.datasets import mnist

In [2]:
NORMALISATION = 255
NUM_DIGITS = 10
PIXELS = 28 * 28
NUM_SAMPLES = 1000

ALPHA = 0.005
ITERATIONS = 100
HIDDEN_SIZE = 40

In [3]:
# Create training set
(x_train, y_train), (x_test, y_test) = mnist.load_data()
train_images = x_train[:NUM_SAMPLES].reshape(NUM_SAMPLES, PIXELS) / NORMALISATION    # Flatten original 3D tensor to 2D

train_labels = np.zeros((len(y_train[:NUM_SAMPLES]), NUM_DIGITS))                      

for i, label in enumerate(y_train[:NUM_SAMPLES]):
    train_labels[i][label] = 1

# Create test set
test_images = x_test.reshape(len(x_test), PIXELS) / NORMALISATION

test_labels = np.zeros((len(y_test), NUM_DIGITS))

for i, label in enumerate(y_test):
    test_labels[i][label] = 1

In [11]:
# ReLU functions
def relu(arr: np.ndarray) -> np.ndarray:
    return np.maximum(0, arr)

def relu2deriv(arr: np.ndarray) -> np.ndarray:
    return (arr >= 0).astype(int)

In [12]:
# Initialise weights
np.random.seed(1)
weights_0_1 = np.random.uniform(-0.1, 0.1, (PIXELS, HIDDEN_SIZE))
weights_1_2 = np.random.uniform(-0.1, 0.1, (HIDDEN_SIZE, NUM_DIGITS))

In [13]:
# Run training loop. It currently processes one sample at a
# time. 

for iteration in range(ITERATIONS): 
    error, correct = (0, 0)

    for i in range(len(train_images)):
        layer_0 = train_images[i:i+1]               # Slicing like this retains shape of array needed for dot product
        layer_1 = relu(np.dot(layer_0,weights_0_1)) 
        layer_2 = np.dot(layer_1,weights_1_2)
        
        error += np.sum((train_labels[i:i+1] - layer_2) ** 2)
        correct += int(np.argmax(layer_2) == np.argmax(train_labels[i:i+1])) 
        
        layer_2_delta = (train_labels[i:i+1] - layer_2)
        layer_1_delta = layer_2_delta.dot(weights_1_2.T) * relu2deriv(layer_1)
        
        weights_1_2 += ALPHA * layer_1.T.dot(layer_2_delta) 
        weights_0_1 += ALPHA * layer_0.T.dot(layer_1_delta)

    sys.stdout.write("\r I:"+str(iteration)+ \
                     " Train-Err:" + str(error/float(len(train_images)))[0:5] +\
                     " Train-Acc:" + str(correct/float(len(train_images))))


 I:99 Train-Err:0.167 Train-Acc:0.983